## Курсова
виконала студентка групи ІС-12 мп

Кравчук Олена

Варіант-9

In [6]:
import findspark
findspark.init()

In [7]:
import pyspark
import random

In [8]:
sc = pyspark.SparkContext(appName="Pi")

In [410]:
from pyspark.sql.functions import isnan, col
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder, MinMaxScaler
from pyspark.ml.classification import NaiveBayes, LogisticRegression, DecisionTreeClassifier, RandomForestClassifier
from pyspark.sql.types import DoubleType, IntegerType
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [376]:
spark = SparkSession\
    .builder\
    .appName('courseWork')\
    .master('local')\
    .getOrCreate()

In [377]:
df = spark.read.format('csv').option('header', True).option('inferSchema', True).option('delimiter',';').load("student-por.csv")
df = df.withColumn(
    'target',
    F.when((F.col("G3")>= 0) & (F.col("G3") <=10), 'low')\
    .when((F.col("G3") > 10) & (F.col("G3") <=13), 'medium')\
    .when((F.col("G3")> 13) & (F.col("G3") <=20), 'high')
)
df.toPandas().head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,target
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,3,4,1,1,3,4,0,11,11,medium
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,3,3,1,1,3,2,9,11,11,medium
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,3,2,2,3,3,6,12,13,12,medium
3,GP,F,15,U,GT3,T,4,2,health,services,...,2,2,1,1,5,0,14,14,14,high
4,GP,F,16,U,GT3,T,3,3,other,other,...,3,2,1,2,5,0,11,13,13,medium


In [378]:
df.groupby('target').count().toPandas().head()

,target,count
0,low,197
1,high,194
2,medium,258


In [379]:
def getMissingValuesDescription(df):
    return {
        col: df.filter((df[col] == "") | (df[col] == 'NA') | df[col].isNull() | isnan(df[col])).count()
        for col in df.columns
    }

In [380]:
getMissingValuesDescription(df)

{'school': 0,
 'sex': 0,
 'age': 0,
 'address': 0,
 'famsize': 0,
 'Pstatus': 0,
 'Medu': 0,
 'Fedu': 0,
 'Mjob': 0,
 'Fjob': 0,
 'reason': 0,
 'guardian': 0,
 'traveltime': 0,
 'studytime': 0,
 'failures': 0,
 'schoolsup': 0,
 'famsup': 0,
 'paid': 0,
 'activities': 0,
 'nursery': 0,
 'higher': 0,
 'internet': 0,
 'romantic': 0,
 'famrel': 0,
 'freetime': 0,
 'goout': 0,
 'Dalc': 0,
 'Walc': 0,
 'health': 0,
 'absences': 0,
 'G1': 0,
 'G2': 0,
 'G3': 0,
 'target': 0}

In [381]:
def getNumericalColumns(df):
    return [
        item[0] for item in df.dtypes 
        if item[1].startswith('int')
    ]

In [382]:
def getCategoricalColumns(df):
    return [
        item[0] for item in df.dtypes 
        if item[1].startswith('string')
    ]

In [383]:
def convertToIntValue(df, columnName):
    df = df.withColumn(columnName, df[columnName].cast(IntegerType()))
    return df

In [384]:
numericalColumns = getNumericalColumns(df)
for col in numericalColumns:
    df = convertToIntValue(df, col)
df.printSchema()

root
 |-- school: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- address: string (nullable = true)
 |-- famsize: string (nullable = true)
 |-- Pstatus: string (nullable = true)
 |-- Medu: integer (nullable = true)
 |-- Fedu: integer (nullable = true)
 |-- Mjob: string (nullable = true)
 |-- Fjob: string (nullable = true)
 |-- reason: string (nullable = true)
 |-- guardian: string (nullable = true)
 |-- traveltime: integer (nullable = true)
 |-- studytime: integer (nullable = true)
 |-- failures: integer (nullable = true)
 |-- schoolsup: string (nullable = true)
 |-- famsup: string (nullable = true)
 |-- paid: string (nullable = true)
 |-- activities: string (nullable = true)
 |-- nursery: string (nullable = true)
 |-- higher: string (nullable = true)
 |-- internet: string (nullable = true)
 |-- romantic: string (nullable = true)
 |-- famrel: integer (nullable = true)
 |-- freetime: integer (nullable = true)
 |-- goout: integer (null

In [385]:
trainDf, testDf = df.randomSplit([0.8, 0.2])

In [386]:
categoricalColumns = getCategoricalColumns(trainDf)
categoricalColumns

['school',
 'sex',
 'address',
 'famsize',
 'Pstatus',
 'Mjob',
 'Fjob',
 'reason',
 'guardian',
 'schoolsup',
 'famsup',
 'paid',
 'activities',
 'nursery',
 'higher',
 'internet',
 'romantic',
 'target']

In [387]:
indexers = [StringIndexer(inputCol=column, outputCol= column+"_index") for column in categoricalColumns]
pipeline = Pipeline(stages=indexers).fit(trainDf)
trainIndexerDf = (pipeline
             .transform(trainDf))
testIndexerDf = (pipeline
             .transform(testDf))

In [388]:
trainIndexerDf = trainIndexerDf.drop(*categoricalColumns)
testIndexerDf = testIndexerDf.drop(*categoricalColumns)

In [389]:
trainIndexerDf.toPandas().head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,guardian_index,schoolsup_index,famsup_index,paid_index,activities_index,nursery_index,higher_index,internet_index,romantic_index,target_index
0,15,1,1,2,4,0,3,1,2,1,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,15,1,1,3,1,1,5,5,5,1,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
2,15,2,2,1,1,0,4,3,1,1,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,15,3,3,2,3,0,4,2,1,2,...,2.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
4,15,3,4,1,3,0,4,3,2,1,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [390]:
categoricalColumns.remove('target')

In [391]:
encoders = [OneHotEncoder(inputCol=col+"_index", outputCol=col+'_vector') for col in categoricalColumns]
pipeline = Pipeline(stages=encoders).fit(trainIndexerDf)
trainIndexerDf = (pipeline
             .transform(trainIndexerDf))
testIndexerDf = (pipeline
             .transform(testIndexerDf))

In [392]:
categoricalColumns = [col+"_index" for col in categoricalColumns]

In [393]:
trainDf = trainIndexerDf.drop(*categoricalColumns)
testDf = testIndexerDf.drop(*categoricalColumns)

In [394]:
trainDf.toPandas().head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,reason_vector,guardian_vector,schoolsup_vector,famsup_vector,paid_vector,activities_vector,nursery_vector,higher_vector,internet_vector,romantic_vector
0,15,1,1,2,4,0,3,1,2,1,...,"(0.0, 1.0, 0.0)","(1.0, 0.0)",(0.0),(1.0),(0.0),(0.0),(1.0),(1.0),(1.0),(1.0)
1,15,1,1,3,1,1,5,5,5,1,...,"(1.0, 0.0, 0.0)","(1.0, 0.0)",(1.0),(0.0),(1.0),(0.0),(1.0),(1.0),(1.0),(0.0)
2,15,2,2,1,1,0,4,3,1,1,...,"(0.0, 0.0, 1.0)","(1.0, 0.0)",(0.0),(1.0),(1.0),(0.0),(1.0),(1.0),(0.0),(1.0)
3,15,3,3,2,3,0,4,2,1,2,...,"(0.0, 0.0, 1.0)","(0.0, 0.0)",(1.0),(1.0),(0.0),(0.0),(1.0),(1.0),(1.0),(0.0)
4,15,3,4,1,3,0,4,3,2,1,...,"(1.0, 0.0, 0.0)","(1.0, 0.0)",(0.0),(1.0),(1.0),(0.0),(1.0),(1.0),(1.0),(1.0)


In [395]:
feature_col = trainDf.columns
feature_col.remove('target_index')
df_assembler = VectorAssembler(inputCols=feature_col, outputCol='features')

In [396]:
trainDf = df_assembler.transform(trainDf)
testDf = df_assembler.transform(testDf)

In [397]:
scaler = MinMaxScaler(inputCol='features', outputCol='scaled_features').fit(trainDf)
trainDf = scaler.transform(trainDf)
testDf = scaler.transform(testDf)

In [398]:
lr = LogisticRegression(labelCol='target_index', \
                featuresCol='scaled_features')
lr = lr.fit(trainDf)
testPredictionsDf1 = lr.transform(testDf)

In [399]:
def getAccuracy(testPred):
    evaluator = MulticlassClassificationEvaluator(labelCol='target_index', predictionCol="prediction", metricName="accuracy")
    testNbaccuracy = evaluator.evaluate(testPred)
    print("Test accuracy = " + str(testNbaccuracy))

In [400]:
def getF1(testPred):
    evaluator = MulticlassClassificationEvaluator(labelCol='target_index', predictionCol="prediction", metricName="f1") 
    testf1 = evaluator.evaluate(testPred)
    print("Test F1 = " + str(testf1))

In [401]:
getAccuracy(testPredictionsDf1)
getF1(testPredictionsDf1)

Test accuracy = 0.8416666666666667
Test F1 = 0.8411277017403492


In [407]:
testPredictionsDf1.toPandas().head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,activities_vector,nursery_vector,higher_vector,internet_vector,romantic_vector,features,scaled_features,rawPrediction,probability,prediction
0,15,1,1,1,2,0,3,3,4,2,...,(1.0),(0.0),(1.0),(1.0),(0.0),"[15.0, 1.0, 1.0, 1.0, 2.0, 0.0, 3.0, 3.0, 4.0,...","(0.0, 0.25, 0.25, 0.0, 0.3333333333333333, 0.0...","[7.617264406446465, 0.8057659051102846, -8.423...","[0.9989000613559097, 0.0010998306722371945, 1....",0.0
1,15,2,4,1,3,0,4,3,2,1,...,(0.0),(1.0),(1.0),(1.0),(1.0),"[15.0, 2.0, 4.0, 1.0, 3.0, 0.0, 4.0, 3.0, 2.0,...","(0.0, 0.5, 1.0, 0.0, 0.6666666666666666, 0.0, ...","[6.116088276785533, 1.3756279966907687, -7.491...","[0.991339798533302, 0.008658981274862192, 1.22...",0.0
2,15,3,4,2,3,0,4,2,2,2,...,(1.0),(1.0),(1.0),(1.0),(0.0),"[15.0, 3.0, 4.0, 2.0, 3.0, 0.0, 4.0, 2.0, 2.0,...","[0.0, 0.75, 1.0, 0.3333333333333333, 0.6666666...","[8.316670483146893, -6.153440698452769, -2.163...","[0.999971385644834, 5.19634687464052e-07, 2.80...",0.0
3,15,1,2,1,2,0,2,3,4,2,...,(1.0),(0.0),(1.0),(0.0),(0.0),"[15.0, 1.0, 2.0, 1.0, 2.0, 0.0, 2.0, 3.0, 4.0,...","(0.0, 0.25, 0.5, 0.0, 0.3333333333333333, 0.0,...","[4.767254585276177, 1.3701769546935836, -6.137...","[0.9675958782256341, 0.032386345162154016, 1.7...",0.0
4,15,2,1,3,3,0,5,2,2,1,...,(0.0),(1.0),(1.0),(1.0),(1.0),"[15.0, 2.0, 1.0, 3.0, 3.0, 0.0, 5.0, 2.0, 2.0,...","[0.0, 0.5, 0.25, 0.6666666666666666, 0.6666666...","[6.620354366900118, -10.181355088738702, 3.561...","[0.9551845905519534, 4.821671343253594e-08, 0....",0.0


In [408]:
tree = DecisionTreeClassifier(labelCol='target_index', \
                featuresCol='features')
tree = tree.fit(trainDf)
testPredictionsDf2 = tree.transform(testDf)

In [409]:
getAccuracy(testPredictionsDf2)
getF1(testPredictionsDf2)

Test accuracy = 1.0
Test F1 = 1.0


In [411]:
forest = RandomForestClassifier(labelCol='target_index', \
                featuresCol='features')
forest = tree.fit(trainDf)
testPredictionsDf3 = forest.transform(testDf)

In [412]:
getAccuracy(testPredictionsDf3)
getF1(testPredictionsDf3)

Test accuracy = 0.9833333333333333
Test F1 = 0.9832561728395062


In [416]:
nb = NaiveBayes(labelCol='target_index', \
                featuresCol='features', \
                 modelType='gaussian')
nb= nb.fit(trainDf)
testPredictionsDf4 = nb.transform(testDf)

In [417]:
getAccuracy(testPredictionsDf4)
getF1(testPredictionsDf4)

Test accuracy = 0.7916666666666666
Test F1 = 0.7863687600644123
